In [1]:
import bayesfast as bf
import numpy as np

In [2]:
D = 64 # number of dims
a = 2.
b = 1.
lower = np.full(D, -5.) # lower bound of the prior
upper = np.full(D, 5.) # upper bound of the prior
bound = np.array((lower, upper)).T
diff = bound[:, 1] - bound[:, 0]
const = np.sum(np.log(diff)) # normalization of the flat prior

def logp(x):
    x2 = x * x
    x2s = np.concatenate((x2[..., -1:], x2, x2[..., :1]), axis=-1)
    return -np.sum((x2s[..., :-2] + x2s[..., 1:-1] - a)**2 / b, axis=-1) - const

def grad(x):
    x2 = x * x
    x2s = np.concatenate((x2[..., -1:], x2, x2[..., :1]), axis=-1)
    return -(2 * (x2s[..., :-2] + x2s[..., 1:-1] - a) / b + 
             2 * (x2s[..., 1:-1] + x2s[..., 2:] - a) / b) * 2 * x

In [3]:
from distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=8, threads_per_worker=1)
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:40691 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 126.70 GB


In [4]:
den = bf.DensityLite(logp=logp, grad=grad, input_size=D, input_scales=bound,
                     hard_bounds=True)
sample_trace = {'n_chain': 8, 'n_iter': 2500, 'n_warmup': 1000}
rec = bf.Recipe(density=den, sample={'sample_trace': sample_trace,
                'random_state': 0}, post={}, client=client)

In [5]:
rec.run()

/global/u2/h/hejia/bayesfast/bayesfast/core/sample.py:104: RuntimeWarning:  CHAIN #6 : divide by zero encountered in log
  warnings.warn(msg[1], msg[0])
/global/u2/h/hejia/bayesfast/bayesfast/core/sample.py:104: RuntimeWarning:  CHAIN #6 : divide by zero encountered in true_divide
  warnings.warn(msg[1], msg[0])
/global/u2/h/hejia/bayesfast/bayesfast/core/sample.py:104: RuntimeWarning:  CHAIN #5 : divide by zero encountered in true_divide
  warnings.warn(msg[1], msg[0])
/global/u2/h/hejia/bayesfast/bayesfast/core/sample.py:104: RuntimeWarning:  CHAIN #5 : divide by zero encountered in log
  warnings.warn(msg[1], msg[0])
/global/u2/h/hejia/bayesfast/bayesfast/core/sample.py:104: RuntimeWarning:  CHAIN #3 : divide by zero encountered in log
  warnings.warn(msg[1], msg[0])
/global/u2/h/hejia/bayesfast/bayesfast/core/sample.py:104: RuntimeWarning:  CHAIN #3 : divide by zero encountered in true_divide
  warnings.warn(msg[1], msg[0])
/global/u2/h/hejia/bayesfast/bayesfast/core/sample.py:104:

 CHAIN #1 : sampling proceeding [ 500 / 2500 ], last 500 samples used 8.85 seconds. (warmup)
 CHAIN #0 : sampling proceeding [ 500 / 2500 ], last 500 samples used 9.05 seconds. (warmup)
 CHAIN #7 : sampling proceeding [ 500 / 2500 ], last 500 samples used 9.18 seconds. (warmup)
 CHAIN #2 : sampling proceeding [ 500 / 2500 ], last 500 samples used 9.30 seconds. (warmup)
 CHAIN #4 : sampling proceeding [ 500 / 2500 ], last 500 samples used 9.83 seconds. (warmup)
 CHAIN #5 : sampling proceeding [ 500 / 2500 ], last 500 samples used 10.40 seconds. (warmup)
 CHAIN #3 : sampling proceeding [ 500 / 2500 ], last 500 samples used 10.43 seconds. (warmup)
 CHAIN #6 : sampling proceeding [ 500 / 2500 ], last 500 samples used 10.88 seconds. (warmup)
 CHAIN #1 : sampling proceeding [ 1000 / 2500 ], last 500 samples used 8.24 seconds. (warmup)
 CHAIN #0 : sampling proceeding [ 1000 / 2500 ], last 500 samples used 8.30 seconds. (warmup)
 CHAIN #2 : sampling proceeding [ 1000 / 2500 ], last 500 samples

/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/glo

 ***** PostStep finished. ***** 



In [6]:
rec.get().logz, rec.get().logz_err # fiducial value: logz = -114.492

(-114.51867958028969, 0.050416277601532916)